# സെമാന്റിക് കർണൽ ഉപകരണ ഉപയോഗ ഉദാഹരണം

ഈ രേഖ സെമാന്റിക് കർണൽ അടിസ്ഥാനമാക്കിയുള്ള ഒരു ഉപകരണം സൃഷ്ടിക്കാൻ ഉപയോഗിക്കുന്ന കോഡിന്റെ അവലോകനവും വിശദീകരണവും നൽകുന്നു, ഇത് Retrieval-Augmented Generation (RAG) വേണ്ടി ChromaDB യുമായി സംയോജിപ്പിക്കുന്നു. ഉദാഹരണം ChromaDB ശേഖരത്തിൽ നിന്ന് യാത്രാ രേഖകൾ തിരികെ ലഭ്യമാക്കുകയും, ഉപയോക്തൃ ചോദ്യങ്ങൾ സെമാന്റിക് തിരച്ചിൽ ഫലങ്ങളാൽ വർദ്ധിപ്പിക്കുകയും, വിശദമായ യാത്രാ ശുപാർശകൾ സ്ട്രീം ചെയ്യുകയും ചെയ്യുന്ന ഒരു AI ഏജന്റ് എങ്ങനെ നിർമ്മിക്കാമെന്ന് കാണിക്കുന്നു.


## പരിസ്ഥിതി ആരംഭിക്കുന്നു


SQLite പതിപ്പ് പരിഹാരം  
നിങ്ങൾക്ക് താഴെ കാണുന്ന പിശക് നേരിടേണ്ടി വന്നാൽ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
നിങ്ങളുടെ നോട്ട്‌ബുക്കിന്റെ തുടക്കത്തിൽ ഈ കോഡ് ബ്ലോക്ക് അൺകമ്മെന്റ് ചെയ്യുക:  


In [ ]:
# %pip install pysqlite3-binary
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
!pip install chromadb

### പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുന്നു  
താഴെ കൊടുത്തിരിക്കുന്ന കോഡ് ആവശ്യമായ പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുന്നു:  


In [ ]:
import json
import os
import chromadb
from typing import Annotated, TYPE_CHECKING

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

if TYPE_CHECKING:
    from chromadb.api.models.Collection import Collection

### സെമാന്റിക് കർണലും AI സേവനവും സൃഷ്ടിക്കൽ

ഒരു സെമാന്റിക് കർണൽ ഉദാഹരണം സൃഷ്ടിക്കുകയും അസിങ്ക്രോണസ് OpenAI ചാറ്റ് കംപ്ലീഷൻ സേവനത്തോടെ ക്രമീകരിക്കുകയും ചെയ്യുന്നു. പ്രതികരണങ്ങൾ സൃഷ്ടിക്കുന്നതിന് സേവനം കർണലിൽ ചേർക്കുന്നു.


In [ ]:
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)


# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### പ്രോംപ്റ്റ് പ്ലഗിൻ നിർവചിക്കുന്നത്

PromptPlugin retrieval context ഉപയോഗിച്ച് ഒരു വർദ്ധിത പ്രോംപ്റ്റ് നിർമ്മിക്കുന്നതിനുള്ള ഫംഗ്ഷൻ നിർവചിക്കുന്ന ഒരു നേറ്റീവ് പ്ലഗിനാണ്.


In [ ]:
class PromptPlugin:

    def __init__(self, collection: "Collection"):
        self.collection = collection

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context."
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide your answer."
        )
    
    @kernel_function(name="retrieve_context", description="Retrieve context from the database.")
    def get_retrieval_context(self, query: str) -> str:
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_entries = []
        if results and results.get("documents") and results["documents"][0]:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_entries.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_entries) if context_entries else "No retrieval context found."

### കാലാവസ്ഥാ വിവരങ്ങൾ പ്ലഗിൻ നിർവചിക്കുന്നു

WeatherInfoPlugin ഒരു നാടൻ പ്ലഗിൻ ആണ്, ഇത് പ്രത്യേക യാത്രാ ലക്ഷ്യസ്ഥലങ്ങളുടെ താപനില വിവരങ്ങൾ നൽകുന്നു.


In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

### ഡെസ്റ്റിനേഷൻസ് ഇൻഫർമേഷൻ പ്ലഗിൻ നിർവചിക്കുന്നത്

DestinationsPlugin ഒരു നാടുവാഴി പ്ലഗിൻ ആണ്, ഇത് പ്രശസ്തമായ യാത്രാ ഡെസ്റ്റിനേഷനുകളെക്കുറിച്ചുള്ള വിശദമായ വിവരങ്ങൾ നൽകുന്നു.


In [ ]:
class DestinationsPlugin:
    # Destination data store with rich details about popular travel locations
    DESTINATIONS = {
        "maldives": {
            "name": "The Maldives",
            "description": "An archipelago of 26 atolls in the Indian Ocean, known for pristine beaches and overwater bungalows.",
            "best_time": "November to April (dry season)",
            "activities": ["Snorkeling", "Diving", "Island hopping", "Spa retreats", "Underwater dining"],
            "avg_cost": "$400-1200 per night for luxury resorts"
        },
        "swiss alps": {
            "name": "The Swiss Alps",
            "description": "Mountain range spanning across Switzerland with picturesque villages and world-class ski resorts.",
            "best_time": "December to March for skiing, June to September for hiking",
            "activities": ["Skiing", "Snowboarding", "Hiking", "Mountain biking", "Paragliding"],
            "avg_cost": "$250-500 per night for alpine accommodations"
        },
        "safari": {
            "name": "African Safari",
            "description": "Wildlife viewing experiences across various African countries including Kenya, Tanzania, and South Africa.",
            "best_time": "June to October (dry season) for optimal wildlife viewing",
            "activities": ["Game drives", "Walking safaris", "Hot air balloon rides", "Cultural village visits"],
            "avg_cost": "$400-800 per person per day for luxury safari packages"
        },
        "bali": {
            "name": "Bali, Indonesia",
            "description": "Island paradise known for lush rice terraces, beautiful temples, and vibrant culture.",
            "best_time": "April to October (dry season)",
            "activities": ["Surfing", "Temple visits", "Rice terrace trekking", "Yoga retreats", "Beach relaxation"],
            "avg_cost": "$100-500 per night depending on accommodation type"
        },
        "santorini": {
            "name": "Santorini, Greece",
            "description": "Stunning volcanic island with white-washed buildings and blue domes overlooking the Aegean Sea.",
            "best_time": "Late April to early November",
            "activities": ["Sunset watching in Oia", "Wine tasting", "Boat tours", "Beach hopping", "Ancient ruins exploration"],
            "avg_cost": "$200-600 per night for caldera view accommodations"
        }
    }

    @kernel_function(
        name="get_destination_info",
        description="Provides detailed information about specific travel destinations."
    )
    def get_destination_info(self, query: str) -> str:
        # Find which destination is being asked about
        query_lower = query.lower()
        matching_destinations = []

        for key, details in DestinationsPlugin.DESTINATIONS.items():
            if key in query_lower or details["name"].lower() in query_lower:
                matching_destinations.append(details)

        if not matching_destinations:
            return (f"User Query: {query}\n\n"
                    f"I couldn't find specific destination information in our database. "
                    f"Please use the general retrieval system for this query.")

        # Format destination information
        destination_info = "\n\n".join([
            f"Destination: {dest['name']}\n"
            f"Description: {dest['description']}\n"
            f"Best time to visit: {dest['best_time']}\n"
            f"Popular activities: {', '.join(dest['activities'])}\n"
            f"Average cost: {dest['avg_cost']}" for dest in matching_destinations
        ])

        return (f"Destination Information:\n{destination_info}\n\n"
                f"User Query: {query}\n\n"
                "Based on the above destination details, provide a helpful response "
                "that addresses the user's query about this location.")

## ChromaDB സജ്ജമാക്കൽ

Retrieval-Augmented Generation സജ്ജമാക്കുന്നതിനായി, സ്ഥിരമായ ChromaDB ക്ലയന്റ് സൃഷ്ടിക്കുകയും `"travel_documents"` എന്ന പേരിലുള്ള ഒരു ശേഖരം സൃഷ്ടിക്കുകയോ (അല്ലെങ്കിൽ നിലവിലുണ്ടെങ്കിൽ അത് തിരിച്ചെടുക്കുകയോ) ചെയ്യുന്നു. ഈ ശേഖരം പിന്നീട് സാമ്പിള്‍ യാത്രാ രേഖകളും മെറ്റാഡാറ്റയും ഉപയോഗിച്ച് പൂരിപ്പിക്കുന്നു.


In [ ]:
collection = chromadb.PersistentClient(path="./chroma_db").create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True,
)

documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours.",
]

collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[DestinationsPlugin(), WeatherInfoPlugin(), PromptPlugin(collection)],
    name="TravelAgent",
    instructions="Answer travel queries using the provided context. If context is provided, do not say 'I have no context for that.'",
)

### സ്ട്രീമിംഗ് ചാറ്റ് ചരിത്രത്തോടെ ഏജന്റ് പ്രവർത്തിപ്പിക്കൽ  
പ്രധാന അസിങ്ക്രോണസ് ലൂപ്പ് സംഭാഷണത്തിനായി ഒരു ചാറ്റ് ചരിത്രം സൃഷ്ടിക്കുന്നു, ഓരോ ഉപയോക്തൃ ഇൻപുട്ടിനും ആദ്യം വർദ്ധിപ്പിച്ച പ്രോംപ്റ്റ് (സിസ്റ്റം സന്ദേശമായി) ചാറ്റ് ചരിത്രത്തിൽ ചേർക്കുന്നു, അതിനാൽ ഏജന്റ് റിട്രീവൽ കോൺടെക്സ്റ്റ് കാണുന്നു. ഉപയോക്തൃ സന്ദേശവും ചേർക്കുന്നു, തുടർന്ന് ഏജന്റ് സ്ട്രീമിംഗ് ഉപയോഗിച്ച് പ്രവർത്തിപ്പിക്കുന്നു. ഔട്ട്പുട്ട് സ്ട്രീമിംഗ് വഴി വരുമ്പോൾ അത് പ്രിന്റ് ചെയ്യുന്നു.


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖയാണ് വിശ്വസനീയമായ ഉറവിടമായി കണക്കാക്കേണ്ടത്. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
